# Example of using nessai

This notebooks shows an example of using `nessai` for Bayesian inference.

For more details about nessai see: https://nessai.readthedocs.io/

If you want to try this example locally `nessai` can be installed using `pip`:

```python
pip install nessai
```

In [ ]:
import numpy as np
from scipy.stats import norm

from nessai.flowsampler import FlowSampler
from nessai.model import Model
from nessai.utils import setup_logger

We're going set up a logger to log the output from `nessai`, this is based on the implementation in `bilby`.

See: https://git.ligo.org/lscsoft/bilby

In [ ]:
output = './output/'
logger = setup_logger(log_level='WARNING')

## Defining the model
 
In this case we're using a simple 2-dimensionl Gaussian since it will run relatively quickly. 

The model must contain names for each of the parameters and their prior bounds as a dictionary with arrays/lists with the mininimum and maximum.

The main functions in the model should be the `log_prior` and l`og_likelihood`. 

The log prior must be able to accept structed arrays of points where each field is one of the names in the model and there are two extra fields which are `logP` and `logL`. The log-likelihood should accept the same inputs as the prior and return the log-likelihood of the given point.

In [ ]:
class GaussianModel(Model):
    """
    A simple two-dimensional Guassian likelihood
    """
    def __init__(self):
        # Names of parameters to sample
        self.names = ['x0', 'x1']
        # Prior bounds for each parameter
        self.bounds = {n: [-10, 10] for n in self.names}

    def log_prior(self, x):
        """
        Returns log of prior given a live point assuming uniforn
        priors on each parameter.
        
        Also checks the points are within the prior bounds.
        """
        log_p = 0.
        # Iterate through each parameter (x and y)
        # since the live points are a structured array we can
        # get each value using just the name
        for n in self.names:
            log_p += (np.log((x[n] >= self.bounds[n][0])
                             & (x[n] <= self.bounds[n][1]))
                      - np.log(self.bounds[n][1] - self.bounds[n][0]))
        return log_p

    def log_likelihood(self, x):
        """
        Returns log likelihood of given live point assuming a Gaussian
        likelihood.
        """
        log_l = 0
        # Use a Guassian logpdf and iterate through the parameters
        for pn in self.names:
            log_l += norm.logpdf(x[pn])
        return log_l

## Configuring `nessai`

There are two aspects to configure in `nessai`: the normalising flow and the actual sampler.

### Configuring the normalisng flow

The normalsing flow that is trained to produce the proposal points is configured with a dictionary that contains the parameters related to training (e.g. learning rate (lr)) and `model_config` for the configuring
the flow itself (neurons, number of trasformations, etc.)

In [ ]:
flow_config = dict(
    max_epochs=50,
    patience=10,
    model_config=dict(
        n_blocks=2,    # Use two transforms
        n_layers=1,    # Use 1 layer in the NN in each transform
        n_neurons=4,   # Use 4 neurons per layer
    )
)


### Configuring the samper

The `FlowSampler` object is used to managed the sampling settings, run and resume the sampler.

In [ ]:
fs = FlowSampler(
    GaussianModel(),           # The model
    output=output,             # Output directory
    nlive=2000,                # Number of live points
    maximum_uninformed=2000,   # Maximum number of iterations before the flow is trained
    flow_config=flow_config,   # Configuration for the flow
    resume=False,              # Don't resume the previous run
    seed=1234                  # Set a random seed, this also seeds `torch`
)

Then run the sampler.

Whilst running the sampler will print statistics that describe it's current state. We normally focus on `dZ` since by default the sampler will stop when `dZ<=0.1`.

Also, notice how p-values are periodically printed. This is an indication of whether the insertion indices are uniformly distributed.

In [ ]:
fs.run()

## Diagnostic plots

The instance of `FlowSampler` contains the actual nested sampler that has some useful methods that we can use to produce some diagnostic plots

### Trace plot

A trace plot is standard way to examine how the sample explored the parameter space.

In [ ]:
fig = fs.ns.plot_trace()

### State plot

This plot shows us the evolution of the sampler by interation

In [ ]:
fig = fs.ns.plot_state()

### Insertion indices

We can also check the insertion indices, they should be approximatly uniform.

In [ ]:
fig = fs.ns.plot_insertion_indices()